## Pages List (Buy)

In [42]:
import requests
import re
from bs4 import BeautifulSoup

pages_list = []
main_page_url = 'https://www.fincaraiz.com.co/finca-raiz/venta/medellin/?ad=30|1||||1||8,9,3,4,22,2,5,7,19,23,21,18,20|||55|5500006||||||||||||||||1|||1||griddate%20desc||||-1||'
page = requests.get(main_page_url)
soup = BeautifulSoup(page.text, 'html.parser')
pattern = re.compile("var totalPages = [\d]*;")
total_pages = ''
for script in soup.find_all("script", type="text/javascript"):
    if(pattern.findall(script.text)):
        total_pages = int((pattern.findall(script.text)[0].split()[-1])[0:-1])
for i in range(1, total_pages + 1):
    pages_list.append('https://www.fincaraiz.com.co/finca-raiz/venta/medellin/?ad=30|'+ str(i) +'||||1||8,9,3,4,22,2,5,7,19,23,21,18,20|||55|5500006||||||||||||||||1|||1||griddate%20desc||||-1||')

## Properties Pages List (Buy)

In [43]:
paginator_url = 'https://www.fincaraiz.com.co/finca-raiz/venta/medellin/?ad=30|1||||1||8,9,3,4,22,2,5,7,19,23,21,18,20|||55|5500006||||||||||||||||1|||1||griddate%20desc||||-1||'
page = requests.get(paginator_url)
soup = BeautifulSoup(page.content, 'html.parser')
properties_number = soup.find(id='lblNumInm').get_text()
division = soup.find(id='divAdverts')
properties_pages_list = []
for i in range(0, 34):
    properties = division.find(id='rowIndex_' + str(i))
    property = ''
    if (i == 0):
        property = properties.find('li', class_='media')
        property = property.find('span').attrs.get('onclick')
    else:
        property = properties.find('li', class_='media').attrs.get('onclick')
    url_property = str(property).replace('javascript:window.location=', '')
    url_property = 'https://www.fincaraiz.com.co' + url_property.replace("'", "")
    properties_pages_list.append(url_property)

## Convert String to JSON

In [48]:
import json
paginator_url = 'https://www.fincaraiz.com.co/aiana-verde/medellin/proyecto-nuevo-det-4837830.aspx?itemid=4837834'
page = requests.get(paginator_url)
soup = BeautifulSoup(page.content, 'html.parser')

pattern = re.compile("var sfAdvert = \{.*\:.*\:.*\};")
json_property = ''
for script in soup.find_all("script", type="text/javascript"):
    if(pattern.findall(script.text)):
        json_property = pattern.findall(script.text)[0].split()[3:]
        json_to_strip = (json_property[-1])[0:-1]
        json_property = json_property[0:-1]
        json_property.append(json_to_strip)
        json_property = " ".join(json_property)

json_finca_raiz = json.loads(json_property)
print (json.dumps(json_finca_raiz, indent = 4, ensure_ascii=False))

{
    "AdvertId": "4837830",
    "AdvertWASId": "4837830",
    "AdvertExternalId": "AIANA VERDE",
    "ClientId": "1658139",
    "ClientName": "EXERGY LATINOAMERICA S.A.S",
    "ClientURL": "constructora-exergy-latinoamerica-sas-1658139.aspx",
    "AdvertCategoryId": "2",
    "AdvertType": "Used",
    "TransactionId": "1",
    "TransactionType": "Venta",
    "Category1Id": "1",
    "Category2Id": "0",
    "Category3Id": "0",
    "Category1": "Proyecto Nuevo",
    "Category2": "",
    "Category3": "",
    "OriginId": "23",
    "Location1Id": "55",
    "Location2Id": "5500006",
    "Location3Id": "5500101",
    "Location4Id": "5502103",
    "Location1": "Antioquia",
    "Location2": "Medellín",
    "Location3": "Centro",
    "Location4": "Centro",
    "Title": "AIANA VERDE",
    "Description": "El centro de la ciudad se renueva, y con él florece un proyecto que le da un nuevo aire al lugar donde nació todo.AIANA VERDE es el primer edificio sostenible del centro de Medellín. Pensado desde

## Property Scraping (Old Properties)

In [45]:
paginator_url = 'https://www.fincaraiz.com.co/apartamento-en-venta/medellin/america-det-4950356.aspx'
page = requests.get(paginator_url)
soup = BeautifulSoup(page.content, 'html.parser')

# Extract Location
property_location = []
location_object = {
    'country': 'Colombia',
    'department': 'Antioquia',
    'city': 'Medellin',
    'sector': json_finca_raiz["Location3"],
    'neighborhood': json_finca_raiz["Neighborhood"],
    'address': json_finca_raiz["Address"],
    'latitude': float(json_finca_raiz["Latitude"]),
    'longitude': float(json_finca_raiz["Longitude"])
}
property_location.append(location_object)

# Extract Property Agency
property_agency = []
agency_object = {
    'id': int(json_finca_raiz["ClientId"]),
    'name': json_finca_raiz["ClientName"],
    'contractType': json_finca_raiz["ContractType"],
    
    'financing': json_finca_raiz["Financing"],
    'schedule': json_finca_raiz["Schedule"]
}
property_agency.append(agency_object)

# Extract Big Features
property_features = []
features_object = {
    'price': json_finca_raiz["Price"],
    'squareMeters': json_finca_raiz['FormatedSurface'][0:-3],
    'rooms': int(json_finca_raiz['Rooms']),
    'bathrooms': int(json_finca_raiz['Baths']),
    'garages': json_finca_raiz['Garages'],
    'privateArea': json_finca_raiz['FormatedLivingArea'][0:-3],
    'constructionArea': json_finca_raiz['FormatedSurface'][0:-3],
    'squareMetersPrice': json_finca_raiz['PriceM2'],
    'stratum': int(json_finca_raiz['Stratum']),
    'condition': json_finca_raiz['Condition'],
    'antiquity': json_finca_raiz['Ages'],
    'floors': json_finca_raiz['Floor'],
    'interiorFloors': int(json_finca_raiz['InteriorFloors']),
    'weather': json_finca_raiz['Weather'],
    'includesAdministration': json_finca_raiz['IncludesAdministration'],
    'admonPrice': float(json_finca_raiz['AdministrationPrice'])
}
property_features.append(features_object)

# Hidden Extra
extra = soup.find('div', id='DivEstrasHidden')
extras = []
for h4 in extra.find_all('h4'):
    extras.append(h4.text)

property_interior_features = []
property_exterior_features = []
property_sector_features = []

for extra in extras:
    if(extra.find('Caracteristicas Interiores') == 0): # Extract Interior Features
        interior_features = soup.find('ul', id='tblInitialInteriores')
        property_interior_features = [li.text for li in interior_features.find_all('li')]
    elif(extra.find('Caracteristicas Exteriores') == 0): # Extract Exterior Features
        exterior_features = soup.find('ul', id='tblInitialExteriores')
        property_exterior_features = [li.text for li in exterior_features.find_all('li')]
    elif(extra.find('Caracteristicas del Sector') == 0): # Extract Sector Features
        sector_features = soup.find('ul', id='tblInitialdelSector')
        property_sector_features = [li.text for li in sector_features.find_all('li')]

## Property Scraping (New Properties)

In [46]:
paginator_url = 'https://www.fincaraiz.com.co/palermo/medellin/proyecto-nuevo-det-2393445.aspx'
page = requests.get(paginator_url)
soup = BeautifulSoup(page.content, 'html.parser')

# Extract Prices
detail_header = soup.find('div', class_='row detailheader')
detail_price = detail_header.find('div', class_='price')
property_min_price = ''
property_max_price = ''
array_prices = []
for price in detail_price.find_all('label'):
    array_prices.append(price.text)
property_min_price = array_prices[0].split()
property_min_price = " ".join(property_min_price)
property_max_price = array_prices[1].split()
property_max_price = " ".join(property_max_price)

# Extract Builder Company
builder_company = []
builder_company_object = {
    'id': int(json_finca_raiz["ClientId"]),
    'builderCompany': json_finca_raiz["ClientName"],
    'contractType': json_finca_raiz["ContractType"],
    'schedule': json_finca_raiz["Schedule"]
}
builder_company.append(builder_company_object)

# Extract Location
property_location = []
location_object = {
    'country': 'Colombia',
    'department': 'Antioquia',
    'city': 'Medellin',
    'sector': json_finca_raiz["Location3"],
    'neighborhood': json_finca_raiz["Neighborhood"],
    'address': json_finca_raiz["Address"],
    'latitude': float(json_finca_raiz["Latitude"]),
    'longitude': float(json_finca_raiz["Longitude"])
}
property_location.append(location_object)

# Estract Big Features
property_features = []
features_object = {
    'minPrice': property_min_price[2:],
    'maxPrice': property_max_price[2:],
    'squareMeters': json_finca_raiz['FormatedSurface'][0:-3],
    'rooms': int(json_finca_raiz['Rooms']),
    'bathrooms': int(json_finca_raiz['Baths']),
    'garages': json_finca_raiz['Garages'],
    'privateArea': json_finca_raiz['FormatedLivingArea'][0:-3],
    'constructionArea': json_finca_raiz['FormatedSurface'][0:-3],
    'squareMetersPrice': float(json_finca_raiz['PriceM2']),
    'stratum': int(json_finca_raiz['Stratum']),
    'condition': json_finca_raiz['Condition'],
    'antiquity': json_finca_raiz['Ages'],
    'floors': json_finca_raiz['Floor'],
    'interiorFloors': int(json_finca_raiz['InteriorFloors']),
    'weather': json_finca_raiz['Weather'],
    'includesAdministration': json_finca_raiz['IncludesAdministration'],
    'admonPrice': float(json_finca_raiz['AdministrationPrice'])
}
property_features.append(features_object)

# Hidden Extra
extra = soup.find('div', id='DivEstrasHidden')
extras = []
for h4 in extra.find_all('h4'):
    extras.append(h4.text)

property_interior_features = []
property_exterior_features = []
property_sector_features = []

for extra in extras:
    if(extra.find('Caracteristicas Interiores') == 0): # Extract Interior Features
        interior_features = soup.find('ul', id='tblInitialInteriores')
        property_interior_features = [li.text for li in interior_features.find_all('li')]
    elif(extra.find('Caracteristicas Exteriores') == 0): # Extract Exterior Features
        exterior_features = soup.find('ul', id='tblInitialExteriores')
        property_exterior_features = [li.text for li in exterior_features.find_all('li')]
    elif(extra.find('Caracteristicas del Sector') == 0): # Extract Sector Features
        sector_features = soup.find('ul', id='tblInitialdelSector')
        property_sector_features = [li.text for li in sector_features.find_all('li')]

# Extract Offers Types
offers_type = soup.find('div', id='typology')
table = offers_type.find('table')
array_offers_type = []
for tr in table.find_all('tr'):
    property = tr.find_all('td')[0].text.split()
    property = " ".join(property)
    offer_type = tr.find_all('td')[1].text.split()
    offer_type = " ".join(offer_type)
    area = tr.find_all('td')[2].text.split()
    area = " ".join(area)[0:-3].replace(",", ".")
    private_area = tr.find_all('td')[3].text.split()
    private_area = " ".join(private_area)[0:-3].replace(",", ".")
    rooms = tr.find_all('td')[4].text.split()
    rooms = " ".join(rooms)
    baths = tr.find_all('td')[5].text.split()
    baths = " ".join(baths)
    price = tr.find_all('td')[6].text.split()
    price = " ".join(price)[2:]
    
    table_object = {
        'property': property,
        'offerType': offer_type,
        'area': area,
        'privateArea': private_area,
        'rooms': rooms,
        'bathrooms': baths,
        'price': price
    }
    array_offers_type.append(table_object)

## Create JSON

In [47]:
import json

# Old Properties to Buy or Rent JSON
old_property_dict = {
    'code': int(json_finca_raiz["AdvertId"]),
    'status': json_finca_raiz["Status"],
    'type': json_finca_raiz["TransactionType"],
    'use': 'Usado',
    'modifyDate': json_finca_raiz["ModifyDate"],
    'location': property_location,
    'propertyAgency': property_agency,
    'description': json_finca_raiz["Description"],
    'features': property_features,
    'interiorFeatures': property_interior_features,
    'exteriorFeatures': property_exterior_features,
    'sectorFeatures': property_sector_features,
}

# New Properties to Buy JSON
new_property_dict = {
    'code': json_finca_raiz["AdvertId"],
    'status': json_finca_raiz["Status"],
    'type': json_finca_raiz["TransactionType"],
    'use': 'Nuevo',
    'modifyDate': json_finca_raiz["ModifyDate"],
    'nameProject': json_finca_raiz["Title"],
    'builderCompany': builder_company,
    'description': json_finca_raiz["Description"],
    'features': property_features,
    'interiorFeatures': property_interior_features,
    'exteriorFeatures': property_exterior_features,
    'sectorFeatures': property_sector_features,
    'offersType': array_offers_type[1:]
}

#print(json.dumps(old_property_dict, indent=4))